<a href="https://colab.research.google.com/github/benMcCarthy87/jaroURLMatcher/blob/main/URLMatcher-v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import io
import numpy as np
from google.colab import files
from google.colab import drive

'''
If the below is True, matches scored 'above average' will be removed from the list to match with, this will speed up the process of matching remaining URLs
This is a bad option if you have few URL changes, or if you have consolidated URLs so there will be multiple matches to your new URL.
'''
remove_urls = True

'''
If the below is True, if a match scored above average, it will return this and not check the rest of the list. If this is false URLs the above remove_urls
option will become irrelevant - URLs won't be removed when found.
'''

match_if_above_ave = True

In [2]:
#Upload URLs (CSV named urls.csv)

uploaded = files.upload()


Saving urls.csv to urls (6).csv


In [3]:
#Create dataframe

df = pd.read_csv(io.BytesIO(uploaded['urls.csv']))
print(df[:10])

                                            old_URLs                                        new_URLs
0      http://www.examplegroup.com/en/group/profile/  http://www.example-group.com/en/group/profile/
1  https://www.examplegroup.com/en/media/mediacon...  http://www.example-group.com/en/group/profile/
2  https://www.examplegroup.com/en/sustainability...  http://www.example-group.com/en/group/profile/
3             https://www.example.com/us/accessories  http://www.example-group.com/en/group/profile/
4         https://www.example.com/us/adicolorapparel  http://www.example-group.com/en/group/profile/
5         https://www.example.com/us/example_outdoor  http://www.example-group.com/en/group/profile/
6                 https://www.example.com/us/apparel  http://www.example-group.com/en/group/profile/
7                https://www.example.com/us/baseball  http://www.example-group.com/en/group/profile/
8              https://www.example.com/us/basketball  http://www.example-group.com/en/group

In [4]:
from math import floor, ceil 

#The Jaro distance is a measure of similarity between two strings. The higher the Jaro distance for two strings is, the more similar the strings are.
#The below function uses Jaro–Winkler similarity where 1 means an exact match and 0 means there is no similarity.

def jaro_distance(s1, s2): 
    if (s1 == s2): 
        return 1.0
    len1 = len(s1) 
    len2 = len(s2) 
    max_dist = floor(max(len1, len2) / 2) - 1
    match = 0 
    hash_s1 = [0] * len(s1) 
    hash_s2 = [0] * len(s2) 
    for i in range(len1): 
        for j in range(max(0, i - max_dist),  
                       min(len2, i + max_dist + 1)): 
            if (s1[i] == s2[j] and hash_s2[j] == 0): 
                hash_s1[i] = 1
                hash_s2[j] = 1
                match += 1
                break
    if (match == 0): 
        return 0.0
    t = 0
    point = 0
    for i in range(len1): 
        if (hash_s1[i]): 
            while (hash_s2[point] == 0): 
                point += 1
            if (s1[i] != s2[point]): 
                point += 1
                t += 1
    t = t//2
    return (match/ len1 + match / len2 + 
            (match - t + 1) / match)/ 3.0


In [5]:
i = 0
all_distances = []
hld = 0

#function finds the best match using the above Jaro–Winkler distance function.

def get_url(urlO):
    hld = 0
    bm = 'NO URL'
    for  urlN in new_urls:
      ld = jaro_distance(str(urlO), str(urlN))
      simil = ld
      if ld > hld:
        hld = ld
        bm = urlN
        if ld > np.mean(all_distances) and len(all_distances) > 10 and len(bm) > 3 and len(new_urls) > 100:
          print('Found above average match')
          if remove_urls == True:
            new_urls.remove(bm)
            print('removed ' + bm)
            print(str(len(new_urls)) + ' left in new URLs')
          return bm
    all_distances.append(hld)
    return bm
     

In [6]:
matched_urls = []
old_urls = df['old_URLs'].tolist()
new_urls = df['new_URLs'].tolist()

#Remove unchanged URLs

old_out = []
matched_out = []
for old in old_urls:
  if old in new_urls:
      old_out.append(old)
      old_urls.remove(old)
      matched_out.append(old)
      new_urls.remove(old)

#Loop through old URLs, using the get_url function to find the bestmatch

for urlO in old_urls:
  bm = get_url(urlO)
  matched_urls.append(bm)
  print(str(urlO) + ' matched with ' + str(bm))
  i += 1
  print(i, end = '')
  print(' of ', end = '')
  print(len(old_urls))


#Create new dataframe, add old urls and matched new ones.

df_Results = pd.DataFrame(old_urls,columns=['old_urls'])
df_Results.insert(1, 'matched_urls', matched_urls, True)



/usr/local/lib/python3.6/dist-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.6/dist-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Streaming output truncated to the last 5000 lines.
289 of 1377
Found above average match
removed https://www.example.com/usa/shop/baseball
1167 left in new URLs
https://www.example.com/us/baseball matched with https://www.example.com/usa/shop/baseball
290 of 1377
Found above average match
removed https://www.example.com/usa/shop/basketball
1166 left in new URLs
https://www.example.com/us/basketball matched with https://www.example.com/usa/shop/basketball
291 of 1377
Found above average match
removed https://www.example.com/usa/shop/best-sellers
1165 left in new URLs
https://www.example.com/us/best_sellers matched with https://www.example.com/usa/shop/best-sellers
292 of 1377
Found above average match
removed https://www.example.com/usa/shop/blog
1164 left in new URLs
https://www.example.com/us/blog matched with https://www.example.com/usa/shop/blog
293 of 1377
Found above average match
removed https://www.example.com/usa/shop/blog/444197
1163 left in new URLs
https://www.example.com/us

In [ ]:
#Mount Google Drive

drive.mount('drive')

In [ ]:
#Save results as CSV in Drive

df_Results.to_csv('URL_Migration_Results.csv')
!cp URL_Migration_Results.csv "drive/My Drive/"